In [2]:
import os
import pymysql
from dotenv import load_dotenv

# .env 파일 로드
load_dotenv()

# 환경 변수에서 값 가져오기
HOST = os.getenv('HOST')
PORT = int(os.getenv('PORT'))
USER = os.getenv('DB_USER')
PASSWD = os.getenv('PASSWD')
DB = os.getenv('DB')

# DB 이름 wordgame으로 연결 시도
try:
    conn = pymysql.connect(host=HOST, port=PORT, user=USER, passwd=PASSWD, db=DB)
    print("데이터베이스에 성공적으로 연결되었습니다.")
except pymysql.MySQLError as e:
    print(f"데이터베이스 연결 오류: {e}")


데이터베이스에 성공적으로 연결되었습니다.


In [3]:
# 테이블 생성 함수
def create_table():
    conn = pymysql.connect(host=HOST, port=PORT, user=USER, passwd=PASSWD, db=DB)
    cursor = conn.cursor()
    create_table_query = """
    CREATE TABLE IF NOT EXISTS wordgame (
        id INT UNSIGNED NOT NULL AUTO_INCREMENT,
        corr_cnt INT(5),
        exe_time VARCHAR(20) NOT NULL,
        reg_date VARCHAR(20),
        irank INT(5),
        PRIMARY KEY(id)
    );
    """
    cursor.execute(create_table_query)
    conn.commit()
    cursor.close()
    conn.close()

# 게임 정보 저장 함수
def save_game_result(corr_cnt, exe_time):
    conn = pymysql.connect(host=HOST, port=PORT, user=USER, passwd=PASSWD, db=DB)
    cursor = conn.cursor()
    reg_date = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # 등수 계산
    cursor.execute("SELECT id, corr_cnt, exe_time FROM wordgame ORDER BY corr_cnt DESC, exe_time ASC")
    results = cursor.fetchall()
    
    rank = 1
    for result in results:
        if result[1] > corr_cnt or (result[1] == corr_cnt and float(result[2]) < float(exe_time)):
            rank += 1
    
    insert_query = """
    INSERT INTO wordgame (corr_cnt, exe_time, reg_date, irank)
    VALUES (%s, %s, %s, %s)
    """
    cursor.execute(insert_query, (corr_cnt, exe_time, reg_date, rank))
    conn.commit()
    cursor.close()
    conn.close()

# 순위 출력 함수
def print_rankings():
    conn = pymysql.connect(host=HOST, port=PORT, user=USER, passwd=PASSWD, db=DB)
    cursor = conn.cursor()
    cursor.execute("SELECT id, corr_cnt, exe_time, reg_date, irank FROM wordgame ORDER BY irank ASC")
    results = cursor.fetchall()
    print("========== 역대 순위 =============")
    for result in results:
        print(result)
    cursor.close()
    conn.close()

# 메인 함수
def main():
    create_table()
    
    # 예시 게임 결과 저장
    save_game_result(5, '14.923')
    save_game_result(5, '16.798')
    save_game_result(4, '18.399')
    save_game_result(3, '14.318')

    # 순위 출력
    print_rankings()

if __name__ == "__main__":
    main()

NameError: name 'datetime' is not defined